In [1]:
import pandas as pd
import numpy as np
import torch
import os
import time
from imageio import imread
from scipy import linalg
import argparse
import sys
from loguru import logger
from tqdm import tqdm
import cv2
sys.path.insert(0, "../../SemanticStyleGAN")

/usrhomes/g013/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Converting Segmenter output and Our Model's output to images

In [2]:
cut_down_mapping_v2 = {
    0: 0,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 1,
    8: 2,
    9: 1,
    10: 1,
    11: 3,
    12: 4,
    13: 5,
    14: 4,
    15: 3,
    16: 3,
    17: 6,
    18: 6,
    19: 7,
    20: 8,
    21: 9,
    22: 9,
    23: 10,
    24: 11,
    25: 12,
    26: 13,
    27: 14,
    28: 14,
    29: 14,
    30: 14,
    31: 14,
    32: 15,
    33: 15,
}

def simplify_image_labels(image,viewable=False):
    new_image = np.zeros(image.shape)
    for k, v in cut_down_mapping_v2.items():
        mask = image == k
        new_image[mask] =  v if not viewable else  ((v)*255)/15
    return new_image

In [43]:
def save_images(img_list):
    for i,img in enumerate(img_list):
        if len(img )==3:
            img = img.permute(1,2,0)
        img = img.squeeze(0)
        if(torch.is_tensor(img)):
             img = (img).numpy()
        cv2.imwrite(f"./data/sample_image_testing_{i}.png",img)

In [51]:
SSG_output="./data/our_model_output.npy"
segmenter_output="./data/segmentation_model_output_semanticp.npy"
#segmenter_output="./data/segmentation_model_output_SSG.npy"
img_dir= "./data/real_segmentation_input.npy"


ssg=torch.from_numpy(np.load(SSG_output))
segmentor = torch.from_numpy(np.load(segmenter_output))
real_img = torch.from_numpy(np.load(img_dir))

ssg_converted= (ssg*255)/15 ## Our SSG model segmentation output
sem_index_pred = segmentor.max(dim=1, keepdim=True)[1] ##
sem_converted= simplify_image_labels(sem_index_pred,False) ## The SEgmenter segmentation

#Print Which Type Of Images
#save_images(ssg_converted)
#save_images(sem_converted)
#save_images(real_img)

In [57]:
ssg.min()

tensor(0.)

## Analyzing Model Input for segmentation 

In [7]:
semantic_paletter_segmentation_input = "./data/segmentation_model_input_semanticp.npy"
ssg_segmentation_input= "./data/segmentation_model_input_SSG.npy"

In [8]:
sp_seg_input=torch.from_numpy(np.load(semantic_paletter_segmentation_input))
ssg_seg_input=torch.from_numpy(np.load(ssg_segmentation_input))


In [14]:
sp_seg_input.min()

tensor(-1.)

In [12]:
ssg_seg_input.max()

tensor(255.)

Finding was that segmentation pallete input range is from [-1 to 1 ]